In [1]:
%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import gridspec
import matplotlib.patheffects as path_effects
from IPython.display import HTML
import datetime as dt
import re

import numpy as np
import pandas as pd

from scipy.stats import gaussian_kde

import sys
sys.path.append("/Users/jtladner/JTL_GDrive/scripts")
import baltic as bt

def convertDate(x,start,end):
    """ Converts calendar dates between given formats """
    return dt.datetime.strftime(dt.datetime.strptime(x,start),end)


def hpd(data, level):
    """
    Return highest posterior density interval from a list,
    given the percent posterior density interval required.
    """
    d = list(data)
    d.sort()

    nData = len(data)
    nIn = int(round(level * nData))
    if nIn < 2 :
        return None
    #raise RuntimeError("Not enough data. N data: %s"%(len(data)))
 
    i = 0
    r = d[i+nIn-1] - d[i]
    for k in range(len(d) - (nIn - 1)) :
        rk = d[k+nIn-1] - d[k]
        if rk < r :
            r = rk
            i = k

    assert 0 <= i <= i+nIn-1 < len(d)
 
    return (d[i], d[i+nIn-1])

def decimalDate(date,fmt="%Y-%m-%d",variable=False,dateSplitter='-'):
    """ Converts calendar dates in specified format to decimal date. """
    
    if variable==True:
        unknowns=date.count(dateSplitter)

        if unknowns==1:
            #date=dateSplitter.join(date.split(dateSplitter)[:-1])
            fmt=dateSplitter.join(fmt.split(dateSplitter)[:-1])
            precision=1/12.0
        elif unknowns==0:
            #date=dateSplitter.join(date.split(dateSplitter)[:-2])
            fmt=dateSplitter.join(fmt.split(dateSplitter)[:-2])
            precision=1.0
        elif unknowns==2:
            precision=None
    else:
        precision=None
    
    #print '>%s<'%(date)
    
    adatetime=dt.datetime.strptime(date,fmt) ## convert to datetime object
    year = adatetime.year ## get year
    boy = dt.datetime(year, 1, 1) ## get beginning of the year
    eoy = dt.datetime(year + 1, 1, 1) ## get beginning of next year
    return (year + ((adatetime - boy).total_seconds() / ((eoy - boy).total_seconds())),precision) ## return fractional year


typeface='Helvetica Neue'
mpl.rcParams['font.weight']=300
mpl.rcParams['axes.labelweight']=300
mpl.rcParams['font.family']=typeface
mpl.rcParams['font.size']=22

In [1]:



%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import gridspec
import matplotlib.patheffects as path_effects
from IPython.display import HTML
import datetime as dt
import re

import numpy as np
import pandas as pd

from scipy.stats import gaussian_kde

import baltic_highestUsed as bt

## For collapsing clades
from matplotlib.patches import Polygon
import matplotlib.patheffects as path_effects

#
#Path for files, change according to the location of your files
dir_path='/Users/jtladner/Documents/GitHub/Manuscripts/2018_Dokubo_LancetID'
tree_path='%s/2018_Dokubo_LancetID_BEAST_20Mbi_medianMCC.tree' % (dir_path)
log_path='/Users/jtladner/JTL_GDrive/MyPapers/SUBMITTED/DuportRoad/BEAST/OT_prior10_200M/run2/LBR_lineage_for_BEAST_ADARreversed_UCLN_sg50_200M_spriors_edit.log'  % (dir_path)

tmrcas={c:[] for c in ["Nov2015"]}
#!!!!
burnin=10000000
for line in open(log_path,'r'):
    if not line.startswith('#'):
        l=line.strip('\n').split('\t')
        if l[0]=='state':
            header=l
            idx={x:i for i,x in enumerate(header) if 'tmrca' in x}
        elif int(l[0])>burnin:
            for ca in tmrcas.keys():
                tmrcas[ca].append(float(l[idx['tmrca(%s)'%(ca)]]))

ll=bt.loadNexus(tree_path, tip_regex='\_([0-9\-]+)$', highestUsed=2015.1205479452055)
fig,ax = plt.subplots(figsize=(10,15),facecolor='w')
branchWidth=2 ## default branch width
cmap=mpl.cm.viridis
order=[k.numName for k in ll.traverse_tree()]

#Members of the LB5 sublineage
LB5 = set(["CDC659_KT587346_LB5_LBR_2014-09-09",
"EM000457_KR817075_LB5_GUI_2014-09-12",
"EM004059_KR817090_LB5_GUI_2014-12-20",
"EM004085_KR817091_LB5_GUI_2014-12-19",
"EM004201_KR817093_LB5_GUI_2014-12-24",
"EM004259_KR817094_LB5_GUI_2014-12-26",
"EM004290_KR817095_LB5_GUI_2014-12-27",
"EM004414_KR817096_LB5_GUI_2015-01-02",
"EM004422_KR817097_LB5_GUI_2015-01-02",
"EM004437_KR817098_LB5_GUI_2015-01-04",
"EM004438_KR817099_LB5_GUI_2015-01-04",
"EM004481_KR817100_LB5_GUI_2015-01-11",
"EM004494_KR817101_LB5_GUI_2015-01-14",
"EM074462_KR817121_LB5_GUI_2014-08-04",
"EM074548_KR817123_LB5_LBR_2014-08-08",
"EM074821_KR817127_LB5_LBR_2014-08-17",
"EM074822_KR817128_LB5_LBR_2014-08-17",
"EM075076_KR817130_LB5_GUI_2014-08-23",
"EM075373_KR817132_LB5_GUI_2014-08-30",
"EM075435_KR817133_LB5_GUI_2014-08-30",
"EM075447_KR817134_LB5_GUI_2014-08-31",
"EM075928_KR817135_LB5_GUI_2014-10-10",
"EM075929_KR817136_LB5_GUI_2014-10-10",
"EM075930_KR817137_LB5_GUI_2014-10-10",
"EM075931_KR817138_LB5_GUI_2014-10-10",
"EM075932_KR817139_LB5_GUI_2014-10-10",
"EM076191_KR817141_LB5_GUI_2014-10-18",
"EM076192_KR817142_LB5_GUI_2014-10-18",
"EM076217_KR817144_LB5_GUI_2014-10-19",
"EM076334_KR817146_LB5_GUI_2014-10-23",
"EM076335_KR817147_LB5_GUI_2014-10-23",
"EM076383_KR817148_LB5_GUI_2014-10-26",
"EM076403_KR817149_LB5_GUI_2014-10-27",
"EM076472_KR817150_LB5_GUI_2014-10-29",
"EM078415_KR817160_LB5_GUI_2014-11-24",
"EM078416_KR817161_LB5_GUI_2014-11-24",
"EM078556_KR817163_LB5_GUI_2014-12-01",
"EM078638_KR817165_LB5_GUI_2014-12-03",
"EM078639_KR817166_LB5_GUI_2014-12-04",
"EM078656_KR817168_LB5_GUI_2014-12-09",
"EM078670_KR817169_LB5_GUI_2014-12-11",
"EM078683_KR817170_LB5_GUI_2014-12-11",
"EM078694_KR817171_LB5_GUI_2014-12-14",
"EM078706_KR817173_LB5_GUI_2014-12-15",
"EM078722_KR817175_LB5_GUI_2014-12-16",
"EM078763_KR817176_LB5_GUI_2014-12-17",
"EM078779_KR817177_LB5_GUI_2014-12-18",
"IP0633_KR534512_LB5_GUI_2014-08-12",
"IP0645_KR534513_LB5_GUI_2014-08-14",
"IP0691_KR534518_LB5_GUI_2014-08-21",
"IP0768_KR534588_LB5_GUI_2014-08-27",
"IP0789_KR534523_LB5_GUI_2014-08-29",
"IP1063_KR534530_LB5_GUI_2014-09-21",
"IP1120_KR534535_LB5_GUI_2014-09-25",
"IP1121_KR534536_LB5_GUI_2014-09-25",
"IP1205_KR534538_LB5_GUI_2014-10-02",
"IP1210_KR534539_LB5_GUI_2014-10-02",
"IP1274_KR534542_LB5_GUI_2014-10-04",
"IP1321_KR534547_LB5_GUI_2014-10-07",
"IP1339_KR534550_LB5_GUI_2014-10-08",
"IP1340_KR534551_LB5_GUI_2014-10-08",
"IP1342_KR534552_LB5_GUI_2014-10-08",
"IP1355_KR534553_LB5_GUI_2014-10-09",
"IP1394_KR534557_LB5_GUI_2014-10-11",
"IP1445_KR534559_LB5_GUI_2014-10-14",
"IP1454_KR534560_LB5_GUI_2014-10-14",
"IP1480_KR534561_LB5_GUI_2014-10-15",
"IP1481_KR534575_LB5_GUI_2014-10-15",
"IP1491_KR534576_LB5_GUI_2014-10-16",
"IP1561_KR534563_LB5_GUI_2014-10-19",
"IP1568_KR534565_LB5_GUI_2014-10-18",
"IP1622_KR534567_LB5_GUI_2014-10-22",
"IP1623_KR534568_LB5_GUI_2014-10-24",
"KP260799_KP260799_LB5_MLI_2014-10-23",
"KP260800_KP260800_LB5_MLI_2014-11-12",
"KP260801_KP260801_LB5_MLI_2014-11-21",
"KP260802_KP260802_LB5_MLI_2014-11-12",
"LIBR0067_KR006943_LB5_LBR_2014-11-06",
"LIBR0118_KT725291_LB5_LBR_2014-11-10",
"LIBR0176_KR006951_LB5_LBR_2014-11-14",
"LIBR0177_KT725279_LB5_LBR_2014-11-14",
"LIBR0284_KT725328_LB5_LBR_2014-11-22",
"LIBR0286_KR006952_LB5_LBR_2014-11-22",
"LIBR0503_KR006956_LB5_LBR_2014-12-10",
"LIBR0505_KR006957_LB5_LBR_2014-12-10",
"LIBR10035_KT725259_LB5_LBR_2014-08-15",
"LIBR10036_KT725299_LB5_LBR_2014-08-10",
"LIBR10040_KT725314_LB5_LBR_2014-08-04",
"LIBR10042_KT725261_LB5_LBR_2014-08-04",
"LIBR10050_KT725270_LB5_LBR_2014-10-01",
"LIBR10055_KT725340_LB5_LBR_2014-09-23",
"LIBR10061_KT725376_LB5_LBR_2014-09-23",
"LIBR10062_KT725359_LB5_LBR_2014-09-23",
"LIBR10076_KT725334_LB5_LBR_2014-09-20",
"LIBR10103_KT725338_LB5_LBR_2014-08-20",
"LIBR10110_KT725387_LB5_LBR_2014-08-20",
"LIBR10130_KT725343_LB5_LBR_2014-09-01",
"LIBR10136_KT725263_LB5_LBR_2014-09-09",
"LIBR10137_KT725385_LB5_LBR_2014-09-09",
"LIBR10144_KT725271_LB5_LBR_2014-09-20",
"LIBR10148_KT725295_LB5_LBR_2014-09-20",
"LIBR10151_KT725317_LB5_LBR_2014-09-09",
"LIBR10154_KT725318_LB5_LBR_2014-09-13",
"LIBR10169_KT725382_LB5_LBR_2014-08-24",
"LIBR10204_KT725281_LB5_LBR_2014-08-27",
"LIBR10224_KT725302_LB5_LBR_2014-09-26",
"LIBR10251_KT725319_LB5_LBR_2014-09-15",
"LIBR10252_KT725266_LB5_LBR_2014-09-15",
"LIBR10261_KT725287_LB5_LBR_2014-10-14",
"LIBR10267_KT725374_LB5_LBR_2014-10-05",
"IndivH_XXXX_XXX_LBR_2014-08",
"LIBR16393-IndivA_XXXX_XXX_2015-11-19"])


plotTree=ll.Objects
for k in plotTree: ## iterate over objects in tree
    x=k.absoluteTime ## 
    y=k.y ## get y position from .drawTree that was run earlier, but could be anything else
    xp=k.parent.absoluteTime ## get x position of current object's parent
    if x==None: ## matplotlib won't plot Nones, like root
        x=0.0
    if xp==None:
        xp=x-0.1

    #Plotting branch
    ec='w'
    c='grey'
    #Plotting horizontal branches
    if k.branchType=='leaf':
        if k.name in LB5: c='red'
    elif set([ll.tipMap[na] for na in k.leaves]).issubset(LB5): c='red'
    ax.plot([xp,x],[y,y],lw=branchWidth,color=c,ls='-',zorder=8)
        
    if isinstance(k,bt.leaf) or k.branchType=='leaf': ## if leaf...
        s=60 ## tip size can be fixed
        if isinstance(k,bt.leaf):  ## if branch is a leaf object
            if k.name.startswith("LIBR16393"):
                print k.height
                ax.scatter(x,y,s=s,facecolor="green",edgecolor='none',zorder=11) ## plot circle for every tip
                ax.scatter(x,y,s=s+0.9*s,facecolor='black',edgecolor='none',zorder=10) ## plot another circle underneath
                trueDate = decimalDate(k.name.split('_')[-1])[0]
                ax.scatter(trueDate,y,s=s,facecolor="green",edgecolor='none',zorder=11) ## plot circle for every tip
                ax.scatter(trueDate,y,s=s+0.9*s,facecolor='black',edgecolor='none',zorder=10) ## plot another circle underneath
                ax.plot([x,trueDate],[y,y],lw=branchWidth,color="green",ls=':',zorder=8)
                #Print inferred reduction in rate
                print "Inferred reduction in rate:", (trueDate-xp)/(x-xp)
            if k.name.startswith("IndivH"):
                ax.scatter(x,y,s=s,facecolor="blue",edgecolor='none',zorder=11) ## plot circle for every tip
                ax.scatter(x,y,s=s+0.9*s,facecolor='black',edgecolor='none',zorder=10) ## plot another circle underneath
        else: ## it's actually a clade object
            clade=plt.Polygon(([x,y-0.001*len(ll.Objects)],[x,y+0.001*len(ll.Objects)],[k.lastAbsoluteTime,y+k.width/2.0],[k.lastAbsoluteTime,y-k.width/2.0]), facecolor = c2, edgecolor='none',zorder=12)
            ax.add_patch(clade)
#             ax.text(2010,y,'%s'%(k.numName), ha='right',va='center', size=12, zorder=13)
            ax.text(2010,y,k.numName.split()[0], ha='right',va='center', size=12, zorder=13)
            #Plot black circle if good posterior support
            if k.traits.has_key('posterior'):
                if k.traits['posterior'] >= 0.90:
                    ax.scatter(x,y,s=15,facecolor='k',edgecolor='none',zorder=13)        
            if k.traits.has_key('location.prob'):
                if float(k.traits['location.prob']) >= 0.90:
                    ax.scatter(x,y,s=80,facecolor='none',edgecolor=c2,zorder=13)        
                else: 
                    ax.text(x-3,y-.2,'%.0f'%(float(k.traits['location.prob'])*100),size=10,color='k',ha='right', va='top')
        
    elif isinstance(k,bt.node) or k.branchType=='node': ## if node...

        #Plot vertical portions of branches
        ax.plot([x,x],[k.children[-1].y,k.children[0].y],lw=branchWidth,color=c,ls='-',zorder=8)

        #Plot black circle if good posterior
        if k.traits.has_key('posterior'):
            if k.traits['posterior'] >= 0.90:
                ax.scatter(x,y,s=15,facecolor='k',edgecolor='none',zorder=13)        


            
#ax.set_ylim(-1,1+len([w for w in plotTree if w.branchType=='leaf']))

#Plot Nov2015 distribution
tmrca_dist=tmrcas["Nov2015"]
switch=np.median(tmrca_dist)
kde=gaussian_kde(tmrca_dist)
hpdLo,hpdUp=hpd(tmrca_dist,0.95)
x_grid=np.linspace(hpdLo,hpdUp,100)
x_grid_plot=[2016.0-ox for ox in x_grid]
print "Nov2015 = %.2f: %.2f - %.2f" % (switch, hpdLo,hpdUp)

y_grid=kde.evaluate(x_grid)
y_grid=y_grid/y_grid.max()

up=[-20+w*20 for w in y_grid]
lo=[-20 for w in y_grid]
ax.fill_between(x_grid_plot,up,lo,facecolor='green',edgecolor='none',alpha=0.5,zorder=1)
ax.plot(x_grid_plot,up,lw=2,color='green')
#ax.plot(x_grid,lo,lw=2,color='grey')

#Plot label
#ax.text(x+8,y,'%s'%(clade_labels[k.traits['clade']]),size=20,color='k',ha='left', va='center')


#xDates=[]
every=1
#xDates+=['%04d-01-01'%(y) for y in range(2014,2016,every)]
fmt='%Y'

xDates = ["2014-%02d-01" % x for x in range(1,13,1)]
xDates += ["2015-%02d-01" % x for x in range(1,13,1)]

ax.set_xlim(decimalDate(xDates[0])[0],2016.0)
#ax.set_xlim(decimalDate(xDates[0])[0],decimalDate(xDates[-1])[0])

#Drawing vertical lines
[ax.axvline(x,color='grey') for x in [2014, 2014.5, 2015,2015.5, 2016]]
#Drawing alternating grey/white vertical blocks (really just drawing the grey blocks)
[ax.axvspan(decimalDate(xDates[x])[0],decimalDate(xDates[x])[0]+(1/12.),facecolor='k',edgecolor='none',alpha=0.04) for x in range(0,len(xDates),2)]

#Set x-axis tick label positions
ax.set_xticks([2014, 2014.5, 2015,2015.5, 2016])
#Set x-axis tick labels
ax.set_xticklabels(['%.1f' % x for x in [2014, 2014.5, 2015,2015.5, 2016]])
#ax.set_xticklabels([convertDate(x,'%Y-%m-%d',fmt) if x.split('-')[1]=='01' else convertDate(x,'%Y-%m-%d','%b') for x in xDates if (int(x.split('-')[1])-1)%every==0])

#Some more formatting
ax.xaxis.tick_bottom()
ax.yaxis.tick_left()

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.tick_params(axis='x',labelsize=14,size=0)
ax.tick_params(axis='y',labelsize=0,size=0)

#plt.savefig('/Users/jtladner/JTL_GDrive/MyPapers/Lassa/BEAST/with2018/L/bestModel/L_UCLN_sg50.png',dpi=200,bbox_inches='tight')
#plt.savefig('/Users/jtladner/JTL_GDrive/MyPapers/Lassa/BEAST/with2018/L/bestModel/L_UCLN_sg50.tiff',dpi=200,bbox_inches='tight')
#plt.savefig('/Users/jtladner/JTL_GDrive/MyPapers/Lassa/BEAST/with2018/L/bestModel/L_UCLN_sg50.pdf',dpi=200,bbox_inches='tight')

plt.show()

TypeError: not all arguments converted during string formatting